In [1]:
!pip install transformers==4.27.2
!pip install datasets==2.20.0
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
 

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [3]:
from huggingface_hub import list_datasets
print(('knkarthick/dialogsum' in [dataset.id for dataset in list_datasets()] ))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


True


# Importing the dataset :

In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [6]:
dataset["train"][1]

{'id': 'train_1',
 'dialogue': "#Person1#: Hello Mrs. Parker, how have you been?\n#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.\n#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.\n#Person2#: What about Rubella and Mumps?\n#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.\n#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!\n#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.",
 'summary': 'Mrs Parker takes Ricky for his vaccines. Dr. Peters checks the record and then gives Ricky a vaccine.',
 'topic': 'vaccines'}

# Inspecting the data

In [7]:
import random
line = "".join("-" for _ in range(100))
for i in range(1,5):
  random_index = random.randint(0, len(dataset["train"]) - 1)
  print(line)
  print(f"Example : {i}")
  print(line)
  print(f" Conversation : \n {dataset['train'][random_index]['dialogue']}")
  print(line)
  print(f"Human Summary :: { dataset['train'][random_index]['summary']} ")




----------------------------------------------------------------------------------------------------
Example : 1
----------------------------------------------------------------------------------------------------
 Conversation : 
 #Person1#: Thank you for agreeing to come in and meet with us today.
#Person2#: Thank you for inviting me.
#Person1#: I can see from your resume you were in charge of sales for Corporate World Unlimited from 1997 to 1999. Can you please tell me about your responsibilities?
#Person2#: Yes. I was in charge of profit and loss for the northwest region of Canada. As a manager, I was directly responsible for a 50 % increase in sales over a four-month period.
#Person1#: What, do you believe, are some of the key attributes that helped you to achieve such an outstanding increase in sales?
#Person2#: I believe persistence and perseverance are key attributes.
----------------------------------------------------------------------------------------------------
Human Summ

In [8]:
model_name= "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# Tokenization

In [11]:
text = "Deep learning models only takes numbers as input"

tokenized_text = tokenizer(text, return_tensors='tf')
print(f"{tokenized_text}")
detokenized_text = tokenizer.decode(
        tokenized_text["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(tokenized_text["input_ids"][0])
print('\nDECODED SENTENCE:')
print(detokenized_text)

{'input_ids': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[9509, 1036, 2250,  163, 1217, 2302,   38, 3785,    1]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 9), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}
ENCODED SENTENCE:
tf.Tensor([9509 1036 2250  163 1217 2302   38 3785    1], shape=(9,), dtype=int32)

DECODED SENTENCE:
Deep learning models only takes numbers as input


In [12]:
tokenized_text['input_ids']

<tf.Tensor: shape=(1, 9), dtype=int32, numpy=
array([[9509, 1036, 2250,  163, 1217, 2302,   38, 3785,    1]],
      dtype=int32)>

In [13]:
random_index = []
import random
for i in range(3):
  index_from = random.randint(0, len(dataset["train"]) - 1)
  random_index.append(index_from)
random_index

[4676, 5796, 12302]

# Testing the model with no prompts(no intructions)

In [14]:
for index in random_index:
  dialogue = dataset["train"][index]["dialogue"]
  human_summary = dataset["train"][index]["summary"]
  input = tokenizer(dialogue, return_tensors = "pt")
  model_summary = tokenizer.decode (
            model.generate(
          input["input_ids"],
          generation_config=generation_config,
      )[0],skip_special_tokens=True

  )
  print(line)
  print(f"Prompt : \n {dialogue}")
  print(line)
  print(f"Human written summary : {human_summary}")
  print(f"Machine generated summary : {model_summary}")
  print(line)
  print(line)

----------------------------------------------------------------------------------------------------
Prompt : 
 #Person1#: Do you shop online often?
#Person2#: Every so often. It is very convenient and the price is lower than in stores.
#Person1#: Are there many things online?
#Person2#: Books, cosmetics, daily necessities, even large electrical appliance can be ordered from online-stores.
#Person1#: Incredible. I really want to survey the on-line virtual supermarket.
#Person2#: Even you can bargain with the owner of one on-line store, although he or she provides'But It Now'prices.
#Person1#: What are the disadvantages of on-line shopping?
#Person2#: Some provide wrong pictures for the goods, so many buyers are deceived.
----------------------------------------------------------------------------------------------------
Human written summary : #Person1# tells #Person2# about the advantages and disadvantages of shopping online.
Machine generated summary : #Person1#: I really want to sur

# Zero shot Inference

In [15]:
for index in random_index:
  dialogue = dataset["train"][index]["dialogue"]
  human_summary = dataset["train"][index]["summary"]
  prompt = f"""
  Summarize the following conversation.

{dialogue}

Summary:
    """
  input = tokenizer(prompt, return_tensors = "pt")
  model_summary = tokenizer.decode (
            model.generate(
          input["input_ids"],
          generation_config=generation_config
      )[0],skip_special_tokens=True

  )
  print(line)
  print(f"Prompt : \n {dialogue}")
  print(line)
  print(f"Human written summary : {human_summary}")
  print(f"Machine generated summary : {model_summary}")
  print(line)
  print(line)

----------------------------------------------------------------------------------------------------
Prompt : 
 #Person1#: Do you shop online often?
#Person2#: Every so often. It is very convenient and the price is lower than in stores.
#Person1#: Are there many things online?
#Person2#: Books, cosmetics, daily necessities, even large electrical appliance can be ordered from online-stores.
#Person1#: Incredible. I really want to survey the on-line virtual supermarket.
#Person2#: Even you can bargain with the owner of one on-line store, although he or she provides'But It Now'prices.
#Person1#: What are the disadvantages of on-line shopping?
#Person2#: Some provide wrong pictures for the goods, so many buyers are deceived.
----------------------------------------------------------------------------------------------------
Human written summary : #Person1# tells #Person2# about the advantages and disadvantages of shopping online.
Machine generated summary : Shop online.
------------------

# Zero shot inference with different prompt

In [16]:
for index in random_index:
  dialogue = dataset["train"][index]["dialogue"]
  human_summary = dataset["train"][index]["summary"]
  prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""
  input = tokenizer(prompt, return_tensors = "pt")
  model_summary = tokenizer.decode (
            model.generate(
          input["input_ids"],
          generation_config=generation_config
      )[0],skip_special_tokens=True

  )
  print(line)
  print(f"Prompt : \n {dialogue}")
  print(line)
  print(f"Human written summary : {human_summary}")
  print(f"Machine generated summary : {model_summary}")
  print(line)
  print(line)

----------------------------------------------------------------------------------------------------
Prompt : 
 #Person1#: Do you shop online often?
#Person2#: Every so often. It is very convenient and the price is lower than in stores.
#Person1#: Are there many things online?
#Person2#: Books, cosmetics, daily necessities, even large electrical appliance can be ordered from online-stores.
#Person1#: Incredible. I really want to survey the on-line virtual supermarket.
#Person2#: Even you can bargain with the owner of one on-line store, although he or she provides'But It Now'prices.
#Person1#: What are the disadvantages of on-line shopping?
#Person2#: Some provide wrong pictures for the goods, so many buyers are deceived.
----------------------------------------------------------------------------------------------------
Human written summary : #Person1# tells #Person2# about the advantages and disadvantages of shopping online.
Machine generated summary : A person is a customer of on-li

# One shot Inference

In [17]:
for index in random_index:
  dialogue = dataset["train"][index]["dialogue"]
  human_summary = dataset["train"][index]["summary"]
  example_index = random.randint(0, len(dataset["train"]) - 1)
  example_dialogue = dataset["train"][example_index]["dialogue"]
  example_human_summary = dataset["train"][example_index]["summary"]
  prompt = f"""

  Dialogue:

  {example_dialogue}

  What was going on?
  {example_human_summary}


  Dialogue:

  {dialogue}

  What was going on?
  """
  input = tokenizer(prompt, return_tensors = "pt")
  model_summary = tokenizer.decode (
            model.generate(
          input["input_ids"],
          generation_config=generation_config
      )[0],skip_special_tokens=True
            )
  print(line)
  print(f"Prompt : \n {dialogue}")
  print(line)
  print(f"Human written summary : {human_summary}")
  print(f"Machine generated summary : {model_summary}")
  print(line)
  print(line)

----------------------------------------------------------------------------------------------------
Prompt : 
 #Person1#: Do you shop online often?
#Person2#: Every so often. It is very convenient and the price is lower than in stores.
#Person1#: Are there many things online?
#Person2#: Books, cosmetics, daily necessities, even large electrical appliance can be ordered from online-stores.
#Person1#: Incredible. I really want to survey the on-line virtual supermarket.
#Person2#: Even you can bargain with the owner of one on-line store, although he or she provides'But It Now'prices.
#Person1#: What are the disadvantages of on-line shopping?
#Person2#: Some provide wrong pictures for the goods, so many buyers are deceived.
----------------------------------------------------------------------------------------------------
Human written summary : #Person1# tells #Person2# about the advantages and disadvantages of shopping online.
Machine generated summary : #Person1: Shopping online is ve

# Few shot Inference

In [19]:
for index in random_index:
  dialogue = dataset["train"][index]["dialogue"]
  human_summary = dataset["train"][index]["summary"]
  one_example_index = random.randint(0, len(dataset["train"]) - 1)
  one_example_dialogue = dataset["train"][example_index]["dialogue"]
  one_example_human_summary = dataset["train"][example_index]["summary"]
  two_example_index = random.randint(0, len(dataset["train"]) - 1)
  two_example_dialogue = dataset["train"][example_index]["dialogue"]
  two_example_human_summary = dataset["train"][example_index]["summary"]
  three_example_index = random.randint(0, len(dataset["train"]) - 1)
  three_example_dialogue = dataset["train"][example_index]["dialogue"]
  three_example_human_summary = dataset["train"][example_index]["summary"]
  prompt = f"""

  Dialogue:

  {one_example_dialogue}

  What was going on?
  {one_example_human_summary}

  Dialogue:

  {two_example_dialogue}

  What was going on?
  {two_example_human_summary}

    Dialogue:

  {three_example_dialogue}

  What was going on?
  {three_example_human_summary}

  Dialogue:

  {dialogue}

  What was going on?
  """
  input = tokenizer(prompt, return_tensors = "pt")
  model_summary = tokenizer.decode (
            model.generate(
          input["input_ids"],
          generation_config=generation_config
      )[0],skip_special_tokens=True
            )
  print(line)
  print(f"Prompt : \n {dialogue}")
  print(line)
  print(f"Human written summary : {human_summary}")
  print(f"Machine generated summary : {model_summary}")
  print(line)
  print(line)

Token indices sequence length is longer than the specified maximum sequence length for this model (910 > 512). Running this sequence through the model will result in indexing errors


----------------------------------------------------------------------------------------------------
Prompt : 
 #Person1#: Do you shop online often?
#Person2#: Every so often. It is very convenient and the price is lower than in stores.
#Person1#: Are there many things online?
#Person2#: Books, cosmetics, daily necessities, even large electrical appliance can be ordered from online-stores.
#Person1#: Incredible. I really want to survey the on-line virtual supermarket.
#Person2#: Even you can bargain with the owner of one on-line store, although he or she provides'But It Now'prices.
#Person1#: What are the disadvantages of on-line shopping?
#Person2#: Some provide wrong pictures for the goods, so many buyers are deceived.
----------------------------------------------------------------------------------------------------
Human written summary : #Person1# tells #Person2# about the advantages and disadvantages of shopping online.
Machine generated summary : #Person1#: Do you shop online o